In [26]:
import os
import glob
import pandas as pd
import openpyxl

In [27]:
NDA_dir = '/Users/xiaoqianxiao/lab/R01/NDA'
version = 'sumission_20260115'
meta_file_name = 'IFOCUSS_SubInfo.csv'
sumission_dir = os.path.join(NDA_dir, version)
result_dir = os.path.join(sumission_dir, 'filled')
meta_file_path = os.path.join(sumission_dir, 'demo', meta_file_name)
df_meta = pd.read_csv(meta_file_path)
df_meta.head()
df_meta.columns
GUID_file_path = os.path.join(sumission_dir, 'demo', 'GUID_NoHIPinfo.csv')
df_GUID = pd.read_csv(GUID_file_path)
df_meta = df_meta.merge(df_GUID[['ID', 'GUID']], on='ID', how='left')
# Create date_of_birth column in yyyy/mm/dd format from MOB, DOB, YOB
df_meta['date_of_birth'] = df_meta.apply(
    lambda row: f"{int(row['YOB']):04d}/{int(row['MOB']):02d}/{int(row['DOB']):02d}" 
    if pd.notna(row['YOB']) and pd.notna(row['MOB']) and pd.notna(row['DOB']) 
    else None, axis=1
)

In [28]:
template_dir = '/Users/xiaoqianxiao/lab/R01/NDA/data_structure_templates'
defination_dir = '/Users/xiaoqianxiao/lab/R01/NDA/data_structure_defination'
approved_dir = os.path.join(template_dir, 'approved')
redcap_file_dir = os.path.join(sumission_dir,'CSV_from_Redcap')
match_file_path = os.path.join(template_dir, 'NDA_REDCap_matches.xlsx')
df_match_names = pd.ExcelFile(match_file_path)
list_questionnaire_names = df_match_names.sheet_names
print(f"Tabs found: {list_questionnaire_names}")

Tabs found: ['ndar_subject01', 'cde_gad701', 'cde_phq901', 'cde_dsm5crossad01', 'cde_whodas01', 'scidv_rv01', 'lsas01_sr', 'lsas01_cr', 'sds01', 'qids01', 'pswq01', 'hars01', 'babs01', 'rumination01']


In [42]:
#helper functions
# Calculate interview_age in months from date_of_birth to interview_date
# Age is rounded to chronological month: 15 days = 0 months, 16 days = 1 month
from datetime import datetime

# Calculate interview_age in months
def calculate_age_months(dob_str, interview_date_str):
    if pd.isna(dob_str) or pd.isna(interview_date_str):
        return None
    try:
        # Parse dates (date_of_birth is in yyyy/mm/dd format)
        dob = pd.to_datetime(dob_str, format='%Y/%m/%d')
        interview_date = pd.to_datetime(interview_date_str)
        
        # Calculate total days difference
        delta = interview_date - dob
        total_days = delta.days
        
        # Calculate base months (years * 12 + month difference)
        years = interview_date.year - dob.year
        months = interview_date.month - dob.month
        base_months = years * 12 + months
        
        # Adjust based on day of month
        # If interview day is earlier than birth day, we're in the previous month
        if interview_date.day < dob.day:
            base_months -= 1
            # Calculate days into the current month
            days_into_month = interview_date.day
        else:
            # Calculate days into the current month from birth day
            days_into_month = interview_date.day - dob.day
        
        # Apply rounding rule: 15 days = 0 months, 16 days = 1 month
        # If days_into_month >= 16, add 1 month
        if days_into_month >= 16:
            base_months += 1
        
        return max(0, base_months)  # Ensure non-negative
    except:
        return None

# Dictionary for phenotype_description mapping
dic_phenotype_description = {
    'HC': 'Matched healthy controls',
    'SAD': 'Patients with Anxiety Disorders',
    'BDD': 'Patients with Body Dysmorphic Disorder disorders',
    'SAD/BDD': 'Patients with Anxiety Disorders and Body Dysmorphic Disorder disorders'
}

# Dictionary for Race 1=American Indian/Alaska Native; 2=Asian; 3=Black or African American; 4=awaiian or Pacific Islander; 5=White; 6=More than one race; 7=Unknown or not reported
dic_race = {
    1: "American Indian/Alaska Native",
    2: "Asian",
    3: "Black or African American",
    4: "Hawaiian or Pacific Islander",
    5: "White",
    6: "More than one race",
    7: "Unknown or not reported"
}

dic_mdd_lp = {
    1:0,
    3:1
}

In [30]:
#1. ndar_subject01
#NDA: phenotype_description: HC=Matched healthy controls; SAD=Patients with anxiety disorders; BDD=Patients with Body Dysmorphic Disorder disorders
#interview_age: Age in months (Age is rounded to chronological month. If the research participant is 15-days-old at time of interview, the appropriate value would be 0 months. If the participant is 16-days-old, the value would be 1 month.)
#twins_study = 'NO'
#sibling_study = 'NO'
#family_study = 'NO'
#sample_taken = 'NO'
#M = Male; F = Female; O=Other; NR = Not reported
questionaire_name = 'ndar_subject01'
results_file = os.path.join(result_dir, f'{questionaire_name}.csv')
template_path = [f for f in glob.glob(os.path.join(approved_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
redcap_file_path = [f for f in glob.glob(os.path.join(redcap_file_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
df_template = pd.read_csv(template_path, header=1)
# Read the first row of template_path with all columns
first_row_template = pd.read_csv(template_path, header=None, nrows=1, names=df_template.columns)
df_redcap = pd.read_csv(redcap_file_path)
df_target_match_names = df_match_names.parse(questionaire_name, usecols=[0, 1])
valid_mappings = df_target_match_names.dropna(subset=['REDCap'])
df = df_template.copy() 
for index, row in valid_mappings.iterrows():
    nda_col = row['NDA_template']  # The destination column name
    redcap_col = row['REDCap']     # The source column name
    if redcap_col in df_redcap.columns:
        df[nda_col] = df_redcap[redcap_col]
# Merge GUID from df_meta to df['subjectkey'] based on matching ID
guid_map = df_meta.set_index('ID')['GUID'].to_dict()
df['src_subject_id'] = df['src_subject_id'].astype(str).str.strip()
df['subjectkey'] = df['src_subject_id'].map(guid_map)
# Merge SEX from df_meta to df['sex'] based on matching ID
sex_map = df_meta.set_index('ID')['SEX'].to_dict()
df['sex'] = df['src_subject_id'].map(sex_map)
df['sex'] = df['sex'].fillna('NR')
# Merge GROUP from df_meta to df['phenotype'] based on matching ID
dob_map = df_meta.set_index('ID')['GROUP'].to_dict()
df['phenotype'] = df['src_subject_id'].map(dob_map)
# Assign phenotype_description based on phenotype using dictionary
df['phenotype_description'] = df['phenotype'].map(dic_phenotype_description)
# Assign race based on race using dictionary
df['race'] = df['race'].map(dic_race)
# Set fixed values for study type fields
df['twins_study'] = 'No'
df['sibling_study'] = 'No'
df['family_study'] = 'No'
df['sample_taken'] = 'No'
# Merge date_of_birth from df_meta into df
dob_map = df_meta.set_index('ID')['date_of_birth'].to_dict()
df['date_of_birth'] = df['src_subject_id'].map(dob_map)
df['interview_age'] = df.apply(
    lambda row: calculate_age_months(row['date_of_birth'], row['interview_date']), 
    axis=1
)
# Convert interview_age to integer
df['interview_age'] = df['interview_age'].astype('Int64')
df = df.astype(object)
df = df.where(pd.notnull(df), None)
# Convert interview_date to 'MM/DD/YYYY' string format
df['interview_date'] = pd.to_datetime(df['interview_date'])
df['interview_date'] = df['interview_date'].dt.strftime('%m/%d/%Y')
# Trim df to only have columns as in df_template, in the exact same order
df = df[df_template.columns.tolist()]
# Add the first row from template_path to df
df = pd.concat([first_row_template, df], ignore_index=True)
df.iloc[0] = df.iloc[0].fillna('')
df.iloc[:1].to_csv(results_file, index=False, header=False, mode='w')
df.iloc[1:].to_csv(results_file, index=False, header=True, mode='a')

/var/folders/63/3j_hstl96w58qx1sdw9czhxr0000gn/T/ipykernel_54878/557835634.py:57: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['interview_date'] = pd.to_datetime(df['interview_date'])


In [31]:
#2. cde_gad701
#NDA:  0 = Not at all; 1 = Several days; 2= More than half the days; 3 = Nearly every day; -9 = Missing.
#gad7_8: In what language did you collect the data? set to 1(English)
questionaire_name = 'cde_gad701'
results_file = os.path.join(result_dir, f'{questionaire_name}.csv')
template_path = [f for f in glob.glob(os.path.join(approved_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
redcap_file_path = [f for f in glob.glob(os.path.join(redcap_file_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
df_template = pd.read_csv(template_path, header=1)
# Read the first row of template_path with all columns
first_row_template = pd.read_csv(template_path, header=None, nrows=1, names=df_template.columns)
df_redcap = pd.read_csv(redcap_file_path)
df_target_match_names = df_match_names.parse(questionaire_name, usecols=[0, 1])
valid_mappings = df_target_match_names.dropna(subset=['REDCap'])
df = df_template.copy() 
for index, row in valid_mappings.iterrows():
    nda_col = row['NDA_template']  # The destination column name
    redcap_col = row['REDCap']     # The source column name
    if redcap_col in df_redcap.columns:
        df[nda_col] = df_redcap[redcap_col]
# Merge GUID from df_meta to df['subjectkey'] based on matching ID
guid_map = df_meta.set_index('ID')['GUID'].to_dict()
df['src_subject_id'] = df['src_subject_id'].astype(str).str.strip()
df['subjectkey'] = df['src_subject_id'].map(guid_map)
df = df.dropna(subset=['subjectkey'])
# Merge SEX from df_meta to df['sex'] based on matching ID
sex_map = df_meta.set_index('ID')['SEX'].to_dict()
df['sex'] = df['src_subject_id'].map(sex_map)
df['sex'] = df['sex'].fillna('NR')
# Merge date_of_birth from df_meta into df
dob_map = df_meta.set_index('ID')['date_of_birth'].to_dict()
df['date_of_birth'] = df['src_subject_id'].map(dob_map)
df['interview_age'] = df.apply(
    lambda row: calculate_age_months(row['date_of_birth'], row['interview_date']), 
    axis=1
)
df = df.dropna(subset=['interview_age'])
# Set gad7_8 to 1 as english
df['gad7_8'] = '1'
# Set GAD7's missing value to -9
gad7_cols = ['gad7_1', 'gad7_2', 'gad7_3', 'gad7_4', 'gad7_5', 'gad7_6', 'gad7_7']
df[gad7_cols] = df[gad7_cols].fillna(-9)
df[gad7_cols] = df[gad7_cols].astype('Int64')
# Convert interview_age to integer
df['interview_age'] = df['interview_age'].astype('Int64')
df = df.astype(object)
df = df.where(pd.notnull(df), None)
# Convert interview_date to 'MM/DD/YYYY' string format
df['interview_date'] = pd.to_datetime(df['interview_date'])
df['interview_date'] = df['interview_date'].dt.strftime('%m/%d/%Y')
# Trim df to only have columns as in df_template, in the exact same order
df = df[df_template.columns.tolist()]
# Add the first row from template_path to df
df = pd.concat([first_row_template, df], ignore_index=True)
df.iloc[0] = df.iloc[0].fillna('')
df.iloc[:1].to_csv(results_file, index=False, header=False, mode='w')
df.iloc[1:].to_csv(results_file, index=False, header=True, mode='a')

/var/folders/63/3j_hstl96w58qx1sdw9czhxr0000gn/T/ipykernel_54878/2321818529.py:48: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['interview_date'] = pd.to_datetime(df['interview_date'])


In [32]:
#3. cde_phq901
#NDA: 0 = Not at all; 1 = Several days; 2 = More than half the days; 3 = Nearly every day; -9 = Missing
#phq9_10: In what language did you collect the data? set to 1(English)
questionaire_name = 'cde_phq901'
results_file = os.path.join(result_dir, f'{questionaire_name}.csv')
template_path = [f for f in glob.glob(os.path.join(approved_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
redcap_file_path = [f for f in glob.glob(os.path.join(redcap_file_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
df_template = pd.read_csv(template_path, header=1)
# Read the first row of template_path with all columns
first_row_template = pd.read_csv(template_path, header=None, nrows=1, names=df_template.columns)
df_redcap = pd.read_csv(redcap_file_path)
df_target_match_names = df_match_names.parse(questionaire_name, usecols=[0, 1])
valid_mappings = df_target_match_names.dropna(subset=['REDCap'])
df = df_template.copy() 
for index, row in valid_mappings.iterrows():
    nda_col = row['NDA_template']  # The destination column name
    redcap_col = row['REDCap']     # The source column name
    if redcap_col in df_redcap.columns:
        df[nda_col] = df_redcap[redcap_col]
# Merge GUID from df_meta to df['subjectkey'] based on matching ID
guid_map = df_meta.set_index('ID')['GUID'].to_dict()
df['src_subject_id'] = df['src_subject_id'].astype(str).str.strip()
df['subjectkey'] = df['src_subject_id'].map(guid_map)
df = df.dropna(subset=['subjectkey'])
# Merge SEX from df_meta to df['sex'] based on matching ID
sex_map = df_meta.set_index('ID')['SEX'].to_dict()
df['sex'] = df['src_subject_id'].map(sex_map)
# Merge date_of_birth from df_meta into df
dob_map = df_meta.set_index('ID')['date_of_birth'].to_dict()
df['date_of_birth'] = df['src_subject_id'].map(dob_map)
df['interview_age'] = df.apply(
    lambda row: calculate_age_months(row['date_of_birth'], row['interview_date']), 
    axis=1
)
df = df.dropna(subset=['interview_age'])
# Set phq9_10 to 1 as english
df['phq9_10'] = '1'
# Set PHQ9's missing value to -9
phq9_cols = ['phq9_1', 'phq9_2', 'phq9_3', 'phq9_4', 'phq9_5', 'phq9_6', 'phq9_7', 'phq9_8', 'phq9_9']
df[phq9_cols] = df[phq9_cols].fillna(-9)
# Convert interview_age to integer
df['interview_age'] = df['interview_age'].astype('Int64')
df[phq9_cols] = df[phq9_cols].astype('Int64')
df = df.astype(object)
df = df.where(pd.notnull(df), None)
# Convert interview_date to 'MM/DD/YYYY' string format
df['interview_date'] = pd.to_datetime(df['interview_date'])
df['interview_date'] = df['interview_date'].dt.strftime('%m/%d/%Y')
# Trim df to only have columns as in df_template, in the exact same order
df = df[df_template.columns.tolist()]
# Add the first row from template_path to df
df = pd.concat([first_row_template, df], ignore_index=True)
df.iloc[0] = df.iloc[0].fillna('')
df.iloc[:1].to_csv(results_file, index=False, header=False, mode='w')
df.iloc[1:].to_csv(results_file, index=False, header=True, mode='a')

/var/folders/63/3j_hstl96w58qx1sdw9czhxr0000gn/T/ipykernel_54878/2071645460.py:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['interview_date'] = pd.to_datetime(df['interview_date'])


In [33]:
df

,subjectkey,src_subject_id,interview_age,interview_date,sex,phq9_1,phq9_2,phq9_3,phq9_4,phq9_5,phq9_6,phq9_7,phq9_8,phq9_9,phq9_10,study
0,cde_phq9,1,,,,,,,,,,,,,,
1,NDAR_INVMY773RAB,301,252,10/24/2024,F,0,0,1,0,1,0,0,0,0,1,baseline_1_arm_2
2,NDAR_INVNN813ZMK,302,235,10/25/2024,F,0,0,0,0,0,0,0,0,0,1,baseline_1_arm_2
3,NDAR_INVXW311FN4,303,217,10/28/2024,M,0,0,0,1,0,1,0,0,0,1,baseline_1_arm_2
4,NDAR_INVGM360PT8,305,257,11/05/2024,F,0,0,0,1,0,0,0,0,0,1,baseline_1_arm_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,NDARRR532GW3,136,300,12/03/2025,F,0,0,1,0,0,0,0,0,0,1,treatment_session_arm_1c
312,NDARAN078FHK,127,259,12/05/2025,M,0,0,0,0,0,0,0,0,0,1,treatment_session_arm_1i
313,NDARAN078FHK,127,259,12/10/2025,M,0,0,0,0,0,0,0,0,0,1,treatment_session_arm_1j
314,NDARAN078FHK,127,260,12/16/2025,M,0,0,0,0,0,0,0,0,0,1,treatment_session_arm_1k


In [34]:
#4. cde_dsm5crossad01
#NDA: 0 = Not at all; 1 = Several days; 2 = More than half the days; 3 = Nearly every day; -9 = Missing
#dsm5crossad_24: In what language did you collect the data? set to 1(English)
questionaire_name = 'cde_dsm5crossad01'
results_file = os.path.join(result_dir, f'{questionaire_name}.csv')
template_path = [f for f in glob.glob(os.path.join(approved_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
redcap_file_path = [f for f in glob.glob(os.path.join(redcap_file_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
df_template = pd.read_csv(template_path, header=1)
# Read the first row of template_path with all columns
first_row_template = pd.read_csv(template_path, header=None, nrows=1, names=df_template.columns)
df_redcap = pd.read_csv(redcap_file_path)
df_target_match_names = df_match_names.parse(questionaire_name, usecols=[0, 1])
valid_mappings = df_target_match_names.dropna(subset=['REDCap'])
df = df_template.copy() 
for index, row in valid_mappings.iterrows():
    nda_col = row['NDA_template']  # The destination column name
    redcap_col = row['REDCap']     # The source column name
    if redcap_col in df_redcap.columns:
        df[nda_col] = df_redcap[redcap_col]
# Merge GUID from df_meta to df['subjectkey'] based on matching ID
guid_map = df_meta.set_index('ID')['GUID'].to_dict()
df['src_subject_id'] = df['src_subject_id'].astype(str).str.strip()
df['subjectkey'] = df['src_subject_id'].map(guid_map)
# Merge SEX from df_meta to df['sex'] based on matching ID
sex_map = df_meta.set_index('ID')['SEX'].to_dict()
df['sex'] = df['src_subject_id'].map(sex_map)
# Merge date_of_birth from df_meta into df
dob_map = df_meta.set_index('ID')['date_of_birth'].to_dict()
df['date_of_birth'] = df['src_subject_id'].map(dob_map)
df['interview_age'] = df.apply(
    lambda row: calculate_age_months(row['date_of_birth'], row['interview_date']), 
    axis=1
)
# Set dsm5crossad_24 to 1 as english
df['dsm5crossad_24'] = '1'
# Set PHQ9's missing value to -9
dsm_cols = [f'dsm5crossad_{i}' for i in range(1, 24)]
df[dsm_cols] = df[dsm_cols].fillna(-9)
# Convert interview_age to integer
df['interview_age'] = df['interview_age'].astype('Int64')
df[dsm_cols] = df[dsm_cols].astype('Int64')
df = df.astype(object)
df = df.where(pd.notnull(df), None)
df = df.dropna(subset=['interview_age', 'interview_date'])
# Convert interview_date to 'MM/DD/YYYY' string format
df['interview_date'] = pd.to_datetime(df['interview_date'])
df['interview_date'] = df['interview_date'].dt.strftime('%m/%d/%Y')
# Trim df to only have columns as in df_template, in the exact same order
df = df[df_template.columns.tolist()]
# Add the first row from template_path to df
df = pd.concat([first_row_template, df], ignore_index=True)
df.iloc[0] = df.iloc[0].fillna('')
df.iloc[:1].to_csv(results_file, index=False, header=False, mode='w')
df.iloc[1:].to_csv(results_file, index=False, header=True, mode='a')

/var/folders/63/3j_hstl96w58qx1sdw9czhxr0000gn/T/ipykernel_54878/437738121.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['interview_date'] = pd.to_datetime(df['interview_date'])


In [35]:
#5. cde_whodas01
#NDA: 0=None; 1=Mild; 2=Moderate; 3=Severe; 4=Extreme or cannot do; -9 = Missing
#whodas_13: In what language did you collect the data? set to 1(English)
questionaire_name = 'cde_whodas01'
results_file = os.path.join(result_dir, f'{questionaire_name}.csv')
template_path = [f for f in glob.glob(os.path.join(approved_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
redcap_file_path = [f for f in glob.glob(os.path.join(redcap_file_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
df_template = pd.read_csv(template_path, header=1)
# Read the first row of template_path with all columns
first_row_template = pd.read_csv(template_path, header=None, nrows=1, names=df_template.columns)
df_redcap = pd.read_csv(redcap_file_path)
df_target_match_names = df_match_names.parse(questionaire_name, usecols=[0, 1])
valid_mappings = df_target_match_names.dropna(subset=['REDCap'])
df = df_template.copy() 
for index, row in valid_mappings.iterrows():
    nda_col = row['NDA_template']  # The destination column name
    redcap_col = row['REDCap']     # The source column name
    if redcap_col in df_redcap.columns:
        df[nda_col] = df_redcap[redcap_col]
# Merge GUID from df_meta to df['subjectkey'] based on matching ID
guid_map = df_meta.set_index('ID')['GUID'].to_dict()
df['src_subject_id'] = df['src_subject_id'].astype(str).str.strip()
df['subjectkey'] = df['src_subject_id'].map(guid_map)
# Merge SEX from df_meta to df['sex'] based on matching ID
sex_map = df_meta.set_index('ID')['SEX'].to_dict()
df['sex'] = df['src_subject_id'].map(sex_map)
# Merge date_of_birth from df_meta into df
dob_map = df_meta.set_index('ID')['date_of_birth'].to_dict()
df['date_of_birth'] = df['src_subject_id'].map(dob_map)
df['interview_age'] = df.apply(
    lambda row: calculate_age_months(row['date_of_birth'], row['interview_date']), 
    axis=1
)
# Set whodas_13 to 1 as english
df['whodas_13'] = '1'
# Set cde_whodas01's missing value to -9
dsm_cols = [f'whodas_{i}' for i in range(1, 13)]
df[dsm_cols] = df[dsm_cols].fillna(-9)
df[dsm_cols] = df[dsm_cols].astype('Int64')
# Convert interview_age to integer
df['interview_age'] = df['interview_age'].astype('Int64')
df = df.astype(object)
df = df.where(pd.notnull(df), None)
df = df.dropna(subset=['interview_age', 'interview_date'])
# Convert interview_date to 'MM/DD/YYYY' string format
df['interview_date'] = pd.to_datetime(df['interview_date'])
df['interview_date'] = df['interview_date'].dt.strftime('%m/%d/%Y')
# Trim df to only have columns as in df_template, in the exact same order
df = df[df_template.columns.tolist()]
# Add the first row from template_path to df
df = pd.concat([first_row_template, df], ignore_index=True)
df.iloc[0] = df.iloc[0].fillna('')
df.iloc[:1].to_csv(results_file, index=False, header=False, mode='w')
df.iloc[1:].to_csv(results_file, index=False, header=True, mode='a')

/var/folders/63/3j_hstl96w58qx1sdw9czhxr0000gn/T/ipykernel_54878/3634995411.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['interview_date'] = pd.to_datetime(df['interview_date'])


In [43]:
#1. SCID 
#scidv_rv01 for now
#scidv_mdd_lp: Major Depressive Disorders - Lifetime Prevalence; 0=Absent; 1=Present; -99 = Missing
#bodydysmorphiclp: Body Dysmorphic Lifetime Prevalence; 1 = absent; 2 = sub-threshold; 3 = threshold; -9 = no data
#scid_p88: Body Dysmorphic Disorder: Past Month; 1 = absent ; 3 = confirmed ; 555= Missing
#scid_sad_lifetime: Social Anxiety Disorder - Lifetime; 0 = Unknown/Inadequate Information; 1 = Absent; 2 = Sub-threshold; 3 = Threshold
#scid_gad_lifetime: Generalized Anxiety Disorder - Lifetime; 0 = Unknown/Inadequate Information; 1 = Absent; 2 = Sub-threshold; 3 = Threshold
questionaire_name = 'scidv_rv01'
results_file = os.path.join(result_dir, f'{questionaire_name}.csv')

# Find file paths
template_path = [f for f in glob.glob(os.path.join(approved_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
defination_path = [f for f in glob.glob(os.path.join(defination_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
redcap_file_path = [f for f in glob.glob(os.path.join(redcap_file_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]

# Load DataFrames
df_template = pd.read_csv(template_path, header=1)
first_row_template = pd.read_csv(template_path, header=None, nrows=1, names=df_template.columns)
df_redcap = pd.read_csv(redcap_file_path)

# Load Mapping
df_target_match_names = df_match_names.parse(questionaire_name, usecols=[0, 1])
valid_mappings = df_target_match_names.dropna(subset=['REDCap'])

# Apply Mappings
df = df_template.copy() 
for index, row in valid_mappings.iterrows():
    nda_col = row['NDA_template'] 
    redcap_col = row['REDCap']     
    if redcap_col in df_redcap.columns:
        df[nda_col] = df_redcap[redcap_col]

# Merge GUID and Basic Info
guid_map = df_meta.set_index('ID')['GUID'].to_dict()
df['src_subject_id'] = df['src_subject_id'].astype(str).str.strip()
df['subjectkey'] = df['src_subject_id'].map(guid_map)

sex_map = df_meta.set_index('ID')['SEX'].to_dict()
df['sex'] = df['src_subject_id'].map(sex_map)

dob_map = df_meta.set_index('ID')['date_of_birth'].to_dict()
df['date_of_birth'] = df['src_subject_id'].map(dob_map)

df['interview_age'] = df.apply(
    lambda row: calculate_age_months(row['date_of_birth'], row['interview_date']), 
    axis=1
)
df['scidv_mdd_lp'] = df['scidv_mdd_lp'].map(dic_mdd_lp)

# --- Manual Fixes (Apply these BEFORE type enforcement) ---
# Set scidv_mdd_lp missing value to -99
df['scidv_mdd_lp'] = df['scidv_mdd_lp'].fillna(-99)
# Set bodydysmorphiclp missing value to -9
df['bodydysmorphiclp'] = df['bodydysmorphiclp'].fillna(-9)
# Set scid_p88 missing value to 555
df['scid_p88'] = df['scid_p88'].fillna(555)


# --- DYNAMIC TYPE ENFORCEMENT ---
# 1. Load the definition file
df_def = pd.read_csv(defination_path)
dtype_map = dict(zip(df_def['ElementName'], df_def['DataType']))

# 2. Iterate and apply types
for col in df.columns:
    if col in dtype_map:
        target_type = str(dtype_map[col])
        if 'Integer' in target_type:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')
        elif 'Float' in target_type:
            df[col] = pd.to_numeric(df[col], errors='coerce')

# Convert interview_age to integer
df['interview_age'] = df['interview_age'].astype('Int64')

# Drop rows missing critical info
df = df.dropna(subset=['interview_age', 'interview_date'])

# Format Date
df['interview_date'] = pd.to_datetime(df['interview_date'])
df['interview_date'] = df['interview_date'].dt.strftime('%m/%d/%Y')

# Trim columns to match template
df = df[df_template.columns.tolist()]

# --- CRITICAL FIX START ---
# Convert to object to allow mixing Strings (Header) and Integers (Data)
# This preserves the "Int64" formatting (no decimals) but allows the concat below to work.
df = df.astype(object)

# Replace 'pd.NA' (from Int64) with None/Empty string so CSV doesn't print '<NA>'
df = df.where(pd.notnull(df), '')
# --- CRITICAL FIX END ---

# Add the first row (headers) from template
df = pd.concat([first_row_template, df], ignore_index=True)

# Now this fillna will work because the column is type 'object', not 'Int64'
df.iloc[0] = df.iloc[0].fillna('')

# Write to CSV
df.iloc[:1].to_csv(results_file, index=False, header=False, mode='w')
df.iloc[1:].to_csv(results_file, index=False, header=True, mode='a')

/var/folders/63/3j_hstl96w58qx1sdw9czhxr0000gn/T/ipykernel_54878/4014138881.py:54: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['bodydysmorphiclp'] = df['bodydysmorphiclp'].fillna(-9)
/var/folders/63/3j_hstl96w58qx1sdw9czhxr0000gn/T/ipykernel_54878/4014138881.py:56: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['scid_p88'] = df['scid_p88'].fillna(555)
/var/folders/63/3j_hstl96w58qx1sdw9czhxr0000gn/T/ipykernel_54878/4014138881.py:80: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ens

In [44]:
#2. lsas01_sr
# #NDA: 
#fear 0::3;-7; -9	0=None; 1=Mild; 2=Moderate; 3=Severe; -7; -9=incomplete
#avoidence 0=Never; 1=Occasionally; 2=Often; 3=Usually; 4=Almost always; -7; -9=Incomplete
questionaire_name = 'lsas01_sr'
results_file = os.path.join(result_dir, f'{questionaire_name}.csv')
template_path = [f for f in glob.glob(os.path.join(approved_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
redcap_file_path = [f for f in glob.glob(os.path.join(redcap_file_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
df_template = pd.read_csv(template_path, header=1)
# Read the first row of template_path with all columns
first_row_template = pd.read_csv(template_path, header=None, nrows=1, names=df_template.columns)
df_redcap = pd.read_csv(redcap_file_path)
df_target_match_names = df_match_names.parse(questionaire_name, usecols=[0, 1])
valid_mappings = df_target_match_names.dropna(subset=['REDCap'])
df = df_template.copy() 
for index, row in valid_mappings.iterrows():
    nda_col = row['NDA_template']  # The destination column name
    redcap_col = row['REDCap']     # The source column name
    if redcap_col in df_redcap.columns:
        df[nda_col] = df_redcap[redcap_col]
# Merge GUID from df_meta to df['subjectkey'] based on matching ID
guid_map = df_meta.set_index('ID')['GUID'].to_dict()
df['src_subject_id'] = df['src_subject_id'].astype(str).str.strip()
df['subjectkey'] = df['src_subject_id'].map(guid_map)
# Merge SEX from df_meta to df['sex'] based on matching ID
sex_map = df_meta.set_index('ID')['SEX'].to_dict()
df['sex'] = df['src_subject_id'].map(sex_map)
# Merge date_of_birth from df_meta into df
dob_map = df_meta.set_index('ID')['date_of_birth'].to_dict()
df['date_of_birth'] = df['src_subject_id'].map(dob_map)
df['interview_age'] = df.apply(
    lambda row: calculate_age_months(row['date_of_birth'], row['interview_date']), 
    axis=1
)
# Convert interview_age to integer
df['interview_age'] = df['interview_age'].astype('Int64')
# Set lsps01_sr's missing value to -9
start_col = 'phonpubf'
end_col = 'resista'
fill_val = -9
cols_to_fix = df.loc[:, start_col:end_col].columns
df[cols_to_fix] = df[cols_to_fix].fillna(fill_val)
df[cols_to_fix] = df[cols_to_fix].astype('Int64')

df = df.astype(object)
df = df.where(pd.notnull(df), None)
df = df.dropna(subset=['interview_age', 'interview_date'])
# Convert interview_date to 'MM/DD/YYYY' string format
df['interview_date'] = pd.to_datetime(df['interview_date'])
df['interview_date'] = df['interview_date'].dt.strftime('%m/%d/%Y')
# Trim df to only have columns as in df_template, in the exact same order
df = df[df_template.columns.tolist()]
# Add the first row from template_path to df
df = pd.concat([first_row_template, df], ignore_index=True)
df.iloc[0] = df.iloc[0].fillna('')
df.iloc[:1].to_csv(results_file, index=False, header=False, mode='w')
df.iloc[1:].to_csv(results_file, index=False, header=True, mode='a')

/var/folders/63/3j_hstl96w58qx1sdw9czhxr0000gn/T/ipykernel_54878/1615961507.py:49: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['interview_date'] = pd.to_datetime(df['interview_date'])


In [45]:
#2. lsas01_sr
# #NDA: 
#fear 0::3;-7; -9	0=None; 1=Mild; 2=Moderate; 3=Severe; -7; -9=incomplete
#avoidence 0=Never; 1=Occasionally; 2=Often; 3=Usually; 4=Almost always; -7; -9=Incomplete
questionaire_name = 'lsas01_sr'
results_file = os.path.join(result_dir, f'{questionaire_name}.csv')
template_path = [f for f in glob.glob(os.path.join(approved_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
redcap_file_path = [f for f in glob.glob(os.path.join(redcap_file_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
df_template = pd.read_csv(template_path, header=1)
# Read the first row of template_path with all columns
first_row_template = pd.read_csv(template_path, header=None, nrows=1, names=df_template.columns)
df_redcap = pd.read_csv(redcap_file_path)
df_target_match_names = df_match_names.parse(questionaire_name, usecols=[0, 1])
valid_mappings = df_target_match_names.dropna(subset=['REDCap'])
df = df_template.copy() 
for index, row in valid_mappings.iterrows():
    nda_col = row['NDA_template']  # The destination column name
    redcap_col = row['REDCap']     # The source column name
    if redcap_col in df_redcap.columns:
        df[nda_col] = df_redcap[redcap_col]
# Merge GUID from df_meta to df['subjectkey'] based on matching ID
guid_map = df_meta.set_index('ID')['GUID'].to_dict()
df['src_subject_id'] = df['src_subject_id'].astype(str).str.strip()
df['subjectkey'] = df['src_subject_id'].map(guid_map)
# Merge SEX from df_meta to df['sex'] based on matching ID
sex_map = df_meta.set_index('ID')['SEX'].to_dict()
df['sex'] = df['src_subject_id'].map(sex_map)
# Merge date_of_birth from df_meta into df
dob_map = df_meta.set_index('ID')['date_of_birth'].to_dict()
df['date_of_birth'] = df['src_subject_id'].map(dob_map)
df['interview_age'] = df.apply(
    lambda row: calculate_age_months(row['date_of_birth'], row['interview_date']), 
    axis=1
)
# Convert interview_age to integer
df['interview_age'] = df['interview_age'].astype('Int64')
# Set lsps01_sr's missing value to -9
start_col = 'phonpubf'
end_col = 'resista'
fill_val = -9
cols_to_fix = df.loc[:, start_col:end_col].columns
df[cols_to_fix] = df[cols_to_fix].fillna(fill_val)
df[cols_to_fix] = df[cols_to_fix].astype('Int64')

df = df.astype(object)
df = df.where(pd.notnull(df), None)
df = df.dropna(subset=['interview_age', 'interview_date'])
# Convert interview_date to 'MM/DD/YYYY' string format
df['interview_date'] = pd.to_datetime(df['interview_date'])
df['interview_date'] = df['interview_date'].dt.strftime('%m/%d/%Y')
# Trim df to only have columns as in df_template, in the exact same order
df = df[df_template.columns.tolist()]
# Add the first row from template_path to df
df = pd.concat([first_row_template, df], ignore_index=True)
df.iloc[0] = df.iloc[0].fillna('')
df.iloc[:1].to_csv(results_file, index=False, header=False, mode='w')
df.iloc[1:].to_csv(results_file, index=False, header=True, mode='a')

/var/folders/63/3j_hstl96w58qx1sdw9czhxr0000gn/T/ipykernel_54878/1615961507.py:49: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['interview_date'] = pd.to_datetime(df['interview_date'])


In [46]:
#2. lsas01_cr
# #NDA: 
#fear 0::3;-7; -9	0=None; 1=Mild; 2=Moderate; 3=Severe; -7; -9=incomplete
#avoidence 0=Never; 1=Occasionally; 2=Often; 3=Usually; 4=Almost always; -7; -9=Incomplete
questionaire_name = 'lsas01_cr'
results_file = os.path.join(result_dir, f'{questionaire_name}.csv')
template_path = [f for f in glob.glob(os.path.join(approved_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
redcap_file_path = [f for f in glob.glob(os.path.join(redcap_file_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
df_template = pd.read_csv(template_path, header=1)
# Read the first row of template_path with all columns
first_row_template = pd.read_csv(template_path, header=None, nrows=1, names=df_template.columns)
df_redcap = pd.read_csv(redcap_file_path)
df_target_match_names = df_match_names.parse(questionaire_name, usecols=[0, 1])
valid_mappings = df_target_match_names.dropna(subset=['REDCap'])
df = df_template.copy() 
for index, row in valid_mappings.iterrows():
    nda_col = row['NDA_template']  # The destination column name
    redcap_col = row['REDCap']     # The source column name
    if redcap_col in df_redcap.columns:
        df[nda_col] = df_redcap[redcap_col]
# Merge GUID from df_meta to df['subjectkey'] based on matching ID
guid_map = df_meta.set_index('ID')['GUID'].to_dict()
df['src_subject_id'] = df['src_subject_id'].astype(str).str.strip()
df['subjectkey'] = df['src_subject_id'].map(guid_map)
# Merge SEX from df_meta to df['sex'] based on matching ID
sex_map = df_meta.set_index('ID')['SEX'].to_dict()
df['sex'] = df['src_subject_id'].map(sex_map)
# Merge date_of_birth from df_meta into df
dob_map = df_meta.set_index('ID')['date_of_birth'].to_dict()
df['date_of_birth'] = df['src_subject_id'].map(dob_map)
df['interview_age'] = df.apply(
    lambda row: calculate_age_months(row['date_of_birth'], row['interview_date']), 
    axis=1
)
# Convert interview_age to integer
df['interview_age'] = df['interview_age'].astype('Int64')
# Set lsps01_sr's missing value to -9
start_col = 'phonpubf'
end_col = 'resista'
fill_val = -9
cols_to_fix = df.loc[:, start_col:end_col].columns
df[cols_to_fix] = df[cols_to_fix].fillna(fill_val)
df[cols_to_fix] = df[cols_to_fix].astype('Int64')

df = df.astype(object)
df = df.where(pd.notnull(df), None)
df = df.dropna(subset=['interview_age', 'interview_date'])
# Convert interview_date to 'MM/DD/YYYY' string format
df['interview_date'] = pd.to_datetime(df['interview_date'])
df['interview_date'] = df['interview_date'].dt.strftime('%m/%d/%Y')
# Trim df to only have columns as in df_template, in the exact same order
df = df[df_template.columns.tolist()]
# Add the first row from template_path to df
df = pd.concat([first_row_template, df], ignore_index=True)
df.iloc[0] = df.iloc[0].fillna('')
df.iloc[:1].to_csv(results_file, index=False, header=False, mode='w')
df.iloc[1:].to_csv(results_file, index=False, header=True, mode='a')

/var/folders/63/3j_hstl96w58qx1sdw9czhxr0000gn/T/ipykernel_54878/3779840579.py:49: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['interview_date'] = pd.to_datetime(df['interview_date'])


In [ ]:
df

,subjectkey,src_subject_id,interview_date,interview_age,sex,dayscons,visit,certcode,phonpubf,phonpuba,...,aasocial_anx,aasocial_avo,aaperform_anx,aaperform_avo,totalanx,totalav,version_form,askqs,askqas,timepoint_label
0,lsps,1,,,,,,,,,...,,,,,,,,,,
1,NDAR_INVNZ644JB4,102,01/09/2025,241,F,None,baseline_1_arm_1,None,0,0,...,None,None,None,None,None,None,None,None,None,baseline_1_arm_1
2,NDAR_INVNZ644JB4,102,01/27/2025,242,F,None,repeat_baseline_arm_1,None,1,0,...,None,None,None,None,None,None,None,None,None,repeat_baseline_arm_1
3,NDAR_INVNZ644JB4,102,02/26/2025,243,F,None,treatment_session_arm_1d,None,1,0,...,None,None,None,None,None,None,None,None,None,treatment_session_arm_1d
4,NDAR_INVNZ644JB4,102,03/17/2025,244,F,None,treatment_session_arm_1h,None,1,0,...,None,None,None,None,None,None,None,None,None,treatment_session_arm_1h
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,NDARZX442CRX,134,12/16/2025,251,M,None,treatment_session_arm_1h,None,2,3,...,None,None,None,None,None,None,None,None,None,treatment_session_arm_1h
82,NDARLB096RHM,135,10/23/2025,253,F,None,baseline_1_arm_1,None,0,0,...,None,None,None,None,None,None,None,None,None,baseline_1_arm_1
83,NDARLB096RHM,135,11/06/2025,253,F,None,repeat_baseline_arm_1,None,0,0,...,None,None,None,None,None,None,None,None,None,repeat_baseline_arm_1
84,NDARLB096RHM,135,12/04/2025,254,F,None,treatment_session_arm_1d,None,0,0,...,None,None,None,None,None,None,None,None,None,treatment_session_arm_1d


In [ ]:
#3. sds01
#NDA: 0=not at all; 1-3=mildly; 4-6=moderately; 7-9=markedly; 10=extremely; -7= I have not worked/studied at all during the past week for reasons unrelated to the disorder
questionaire_name = 'sds01'
results_file = os.path.join(result_dir, f'{questionaire_name}.csv')
template_path = [f for f in glob.glob(os.path.join(approved_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
redcap_file_path = [f for f in glob.glob(os.path.join(redcap_file_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
df_template = pd.read_csv(template_path, header=1)
# Read the first row of template_path with all columns
first_row_template = pd.read_csv(template_path, header=None, nrows=1, names=df_template.columns)
df_redcap = pd.read_csv(redcap_file_path)
df_target_match_names = df_match_names.parse(questionaire_name, usecols=[0, 1])
valid_mappings = df_target_match_names.dropna(subset=['REDCap'])
df = df_template.copy() 
for index, row in valid_mappings.iterrows():
    nda_col = row['NDA_template']  # The destination column name
    redcap_col = row['REDCap']     # The source column name
    if redcap_col in df_redcap.columns:
        df[nda_col] = df_redcap[redcap_col]
# Merge GUID from df_meta to df['subjectkey'] based on matching ID
guid_map = df_meta.set_index('ID')['GUID'].to_dict()
df['src_subject_id'] = df['src_subject_id'].astype(str).str.strip()
df['subjectkey'] = df['src_subject_id'].map(guid_map)
# Merge SEX from df_meta to df['sex'] based on matching ID
sex_map = df_meta.set_index('ID')['SEX'].to_dict()
df['sex'] = df['src_subject_id'].map(sex_map)
# Merge date_of_birth from df_meta into df
dob_map = df_meta.set_index('ID')['date_of_birth'].to_dict()
df['date_of_birth'] = df['src_subject_id'].map(dob_map)
df['interview_age'] = df.apply(
    lambda row: calculate_age_months(row['date_of_birth'], row['interview_date']), 
    axis=1
)
# Convert interview_age to integer
df['interview_age'] = df['interview_age'].astype('Int64')

df = df.astype(object)
df = df.where(pd.notnull(df), None)
df = df.dropna(subset=['interview_age', 'interview_date'])
# Convert interview_date to 'MM/DD/YYYY' string format
df['interview_date'] = pd.to_datetime(df['interview_date'])
df['interview_date'] = df['interview_date'].dt.strftime('%m/%d/%Y')
# Trim df to only have columns as in df_template, in the exact same order
df = df[df_template.columns.tolist()]
# Add the first row from template_path to df
df = pd.concat([first_row_template, df], ignore_index=True)
df.iloc[0] = df.iloc[0].fillna('')
df.iloc[:1].to_csv(results_file, index=False, header=False, mode='w')
df.iloc[1:].to_csv(results_file, index=False, header=True, mode='a')

/var/folders/63/3j_hstl96w58qx1sdw9czhxr0000gn/T/ipykernel_54878/4063545434.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['interview_date'] = pd.to_datetime(df['interview_date'])


In [ ]:
#4. imaging

In [52]:
#5. qids01
#NDA: 0::3; 999=missing
questionaire_name = 'qids01'
results_file = os.path.join(result_dir, f'{questionaire_name}.csv')
template_path = [f for f in glob.glob(os.path.join(approved_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
redcap_file_path = [f for f in glob.glob(os.path.join(redcap_file_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
df_template = pd.read_csv(template_path, header=1)
# Read the first row of template_path with all columns
first_row_template = pd.read_csv(template_path, header=None, nrows=1, names=df_template.columns)
df_redcap = pd.read_csv(redcap_file_path)
df_target_match_names = df_match_names.parse(questionaire_name, usecols=[0, 1])
valid_mappings = df_target_match_names.dropna(subset=['REDCap'])
df = df_template.copy() 
for index, row in valid_mappings.iterrows():
    nda_col = row['NDA_template']  # The destination column name
    redcap_col = row['REDCap']     # The source column name
    if redcap_col in df_redcap.columns:
        df[nda_col] = df_redcap[redcap_col]
# Merge GUID from df_meta to df['subjectkey'] based on matching ID
guid_map = df_meta.set_index('ID')['GUID'].to_dict()
df['src_subject_id'] = df['src_subject_id'].astype(str).str.strip()
df['subjectkey'] = df['src_subject_id'].map(guid_map)
# Merge SEX from df_meta to df['sex'] based on matching ID
sex_map = df_meta.set_index('ID')['SEX'].to_dict()
df['sex'] = df['src_subject_id'].map(sex_map)
# Merge date_of_birth from df_meta into df
dob_map = df_meta.set_index('ID')['date_of_birth'].to_dict()
df['date_of_birth'] = df['src_subject_id'].map(dob_map)
df['interview_age'] = df.apply(
    lambda row: calculate_age_months(row['date_of_birth'], row['interview_date']), 
    axis=1
)
# Convert interview_age to integer
df['interview_age'] = df['interview_age'].astype('Int64')
# Set lsps01_sr's missing value to -9
start_col = 'vsoin'
end_col = 'vagit'
fill_val = 999
cols_to_fix = df.loc[:, start_col:end_col].columns
df[cols_to_fix] = df[cols_to_fix].fillna(fill_val)
df[cols_to_fix] = df[cols_to_fix].astype('Int64')
df['qvtot'] = df['qvtot'].astype('Int64')

df = df.astype(object)
df = df.where(pd.notnull(df), None)
df = df.dropna(subset=['interview_age', 'interview_date'])
# Convert interview_date to 'MM/DD/YYYY' string format
df['interview_date'] = pd.to_datetime(df['interview_date'])
df['interview_date'] = df['interview_date'].dt.strftime('%m/%d/%Y')
# Trim df to only have columns as in df_template, in the exact same order
df = df[df_template.columns.tolist()]
# Add the first row from template_path to df
df = pd.concat([first_row_template, df], ignore_index=True)
df.iloc[0] = df.iloc[0].fillna('')
df.iloc[:1].to_csv(results_file, index=False, header=False, mode='w')
df.iloc[1:].to_csv(results_file, index=False, header=True, mode='a')

/var/folders/63/3j_hstl96w58qx1sdw9czhxr0000gn/T/ipykernel_54878/2744229511.py:48: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['interview_date'] = pd.to_datetime(df['interview_date'])


In [53]:
#6. pswq01
#NDA: 5= Not at all typical of me; 1= Very typical of me
#pswq_total, 99
#110	treatment_session_arm_1f
questionaire_name = 'pswq01'
results_file = os.path.join(result_dir, f'{questionaire_name}.csv')
template_path = [f for f in glob.glob(os.path.join(approved_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
redcap_file_path = [f for f in glob.glob(os.path.join(redcap_file_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
df_template = pd.read_csv(template_path, header=1)
# Read the first row of template_path with all columns
first_row_template = pd.read_csv(template_path, header=None, nrows=1, names=df_template.columns)
df_redcap = pd.read_csv(redcap_file_path)
df_target_match_names = df_match_names.parse(questionaire_name, usecols=[0, 1])
valid_mappings = df_target_match_names.dropna(subset=['REDCap'])
df = df_template.copy() 
for index, row in valid_mappings.iterrows():
    nda_col = row['NDA_template']  # The destination column name
    redcap_col = row['REDCap']     # The source column name
    if redcap_col in df_redcap.columns:
        df[nda_col] = df_redcap[redcap_col]
# Merge GUID from df_meta to df['subjectkey'] based on matching ID
guid_map = df_meta.set_index('ID')['GUID'].to_dict()
df['src_subject_id'] = df['src_subject_id'].astype(str).str.strip()
df['subjectkey'] = df['src_subject_id'].map(guid_map)
# Merge SEX from df_meta to df['sex'] based on matching ID
sex_map = df_meta.set_index('ID')['SEX'].to_dict()
df['sex'] = df['src_subject_id'].map(sex_map)
# Merge date_of_birth from df_meta into df
dob_map = df_meta.set_index('ID')['date_of_birth'].to_dict()
df['date_of_birth'] = df['src_subject_id'].map(dob_map)
df['interview_age'] = df.apply(
    lambda row: calculate_age_months(row['date_of_birth'], row['interview_date']), 
    axis=1
)
# Convert interview_age to integer
df['interview_age'] = df['interview_age'].astype('Int64')
# Set lsps01_sr's missing value to -9
start_col = 'pswq1'
end_col = 'pswq_total'
fill_val = 99
cols_to_fix = df.loc[:, start_col:end_col].columns
df[cols_to_fix] = df[cols_to_fix].fillna(fill_val)
df[cols_to_fix] = df[cols_to_fix].astype('Int64')
df['pswq_total'] = df['pswq_total'].astype('float')

df = df.astype(object)
df = df.where(pd.notnull(df), None)
df = df.dropna(subset=['interview_age', 'interview_date'])
# Convert interview_date to 'MM/DD/YYYY' string format
df['interview_date'] = pd.to_datetime(df['interview_date'])
df['interview_date'] = df['interview_date'].dt.strftime('%m/%d/%Y')
# Trim df to only have columns as in df_template, in the exact same order
df = df[df_template.columns.tolist()]
# Add the first row from template_path to df
df = pd.concat([first_row_template, df], ignore_index=True)
df.iloc[0] = df.iloc[0].fillna('')
df.iloc[:1].to_csv(results_file, index=False, header=False, mode='w')
df.iloc[1:].to_csv(results_file, index=False, header=True, mode='a')


/var/folders/63/3j_hstl96w58qx1sdw9czhxr0000gn/T/ipykernel_54878/3314267707.py:50: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['interview_date'] = pd.to_datetime(df['interview_date'])


In [55]:
#7. hars01
#NDA: 0::4
questionaire_name = 'hars01'
results_file = os.path.join(result_dir, f'{questionaire_name}.csv')
template_path = [f for f in glob.glob(os.path.join(approved_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
redcap_file_path = [f for f in glob.glob(os.path.join(redcap_file_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
df_template = pd.read_csv(template_path, header=1)
# Read the first row of template_path with all columns
first_row_template = pd.read_csv(template_path, header=None, nrows=1, names=df_template.columns)
df_redcap = pd.read_csv(redcap_file_path)
df_target_match_names = df_match_names.parse(questionaire_name, usecols=[0, 1])
valid_mappings = df_target_match_names.dropna(subset=['REDCap'])
df = df_template.copy() 
for index, row in valid_mappings.iterrows():
    nda_col = row['NDA_template']  # The destination column name
    redcap_col = row['REDCap']     # The source column name
    if redcap_col in df_redcap.columns:
        df[nda_col] = df_redcap[redcap_col]
# Merge GUID from df_meta to df['subjectkey'] based on matching ID
guid_map = df_meta.set_index('ID')['GUID'].to_dict()
df['src_subject_id'] = df['src_subject_id'].astype(str).str.strip()
df['subjectkey'] = df['src_subject_id'].map(guid_map)
# Merge SEX from df_meta to df['sex'] based on matching ID
sex_map = df_meta.set_index('ID')['SEX'].to_dict()
df['sex'] = df['src_subject_id'].map(sex_map)
# Merge date_of_birth from df_meta into df
dob_map = df_meta.set_index('ID')['date_of_birth'].to_dict()
df['date_of_birth'] = df['src_subject_id'].map(dob_map)
df['interview_age'] = df.apply(
    lambda row: calculate_age_months(row['date_of_birth'], row['interview_date']), 
    axis=1
)
# Convert interview_age to integer
df['interview_age'] = df['interview_age'].astype('Int64')
start_col = 'ham_a_q1'
end_col = 'ham_a_q14'
cols_to_fix = df.loc[:, start_col:end_col].columns
df[cols_to_fix] = df[cols_to_fix].astype('Int64')


df = df.astype(object)
df = df.where(pd.notnull(df), None)
df = df.dropna(subset=['interview_age', 'interview_date'])
# Convert interview_date to 'MM/DD/YYYY' string format
df['interview_date'] = pd.to_datetime(df['interview_date'])
df['interview_date'] = df['interview_date'].dt.strftime('%m/%d/%Y')
# Trim df to only have columns as in df_template, in the exact same order
df = df[df_template.columns.tolist()]
# Add the first row from template_path to df
df = pd.concat([first_row_template, df], ignore_index=True)
df.iloc[0] = df.iloc[0].fillna('')
df.iloc[:1].to_csv(results_file, index=False, header=False, mode='w')
df.iloc[1:].to_csv(results_file, index=False, header=True, mode='a')

/var/folders/63/3j_hstl96w58qx1sdw9czhxr0000gn/T/ipykernel_54878/1028507069.py:45: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['interview_date'] = pd.to_datetime(df['interview_date'])


In [57]:
#8. babs01
#NDA: 0::4
#babs_total Sum of Questions 1 - 6
questionaire_name = 'babs01'
results_file = os.path.join(result_dir, f'{questionaire_name}.csv')
template_path = [f for f in glob.glob(os.path.join(approved_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
redcap_file_path = [f for f in glob.glob(os.path.join(redcap_file_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
df_template = pd.read_csv(template_path, header=1)
# Read the first row of template_path with all columns
first_row_template = pd.read_csv(template_path, header=None, nrows=1, names=df_template.columns)
df_redcap = pd.read_csv(redcap_file_path)
df_target_match_names = df_match_names.parse(questionaire_name, usecols=[0, 1])
valid_mappings = df_target_match_names.dropna(subset=['REDCap'])
df = df_template.copy() 
for index, row in valid_mappings.iterrows():
    nda_col = row['NDA_template']  # The destination column name
    redcap_col = row['REDCap']     # The source column name
    if redcap_col in df_redcap.columns:
        df[nda_col] = df_redcap[redcap_col]
# Merge GUID from df_meta to df['subjectkey'] based on matching ID
guid_map = df_meta.set_index('ID')['GUID'].to_dict()
df['src_subject_id'] = df['src_subject_id'].astype(str).str.strip()
df['subjectkey'] = df['src_subject_id'].map(guid_map)
# Merge SEX from df_meta to df['sex'] based on matching ID
sex_map = df_meta.set_index('ID')['SEX'].to_dict()
df['sex'] = df['src_subject_id'].map(sex_map)
# Merge date_of_birth from df_meta into df
dob_map = df_meta.set_index('ID')['date_of_birth'].to_dict()
df['date_of_birth'] = df['src_subject_id'].map(dob_map)
df['interview_age'] = df.apply(
    lambda row: calculate_age_months(row['date_of_birth'], row['interview_date']), 
    axis=1
)
# Convert interview_age to integer
df['interview_age'] = df['interview_age'].astype('Int64')
df['idor'] = df['idor'].astype('Int64')

df = df.astype(object)
df = df.where(pd.notnull(df), None)
df = df.dropna(subset=['interview_age', 'interview_date'])
# Convert interview_date to 'MM/DD/YYYY' string format
df['interview_date'] = pd.to_datetime(df['interview_date'])
df['interview_date'] = df['interview_date'].dt.strftime('%m/%d/%Y')
# Trim df to only have columns as in df_template, in the exact same order
df = df[df_template.columns.tolist()]
# Add the first row from template_path to df
df = pd.concat([first_row_template, df], ignore_index=True)
df.iloc[0] = df.iloc[0].fillna('')
df.iloc[:1].to_csv(results_file, index=False, header=False, mode='w')
df.iloc[1:].to_csv(results_file, index=False, header=True, mode='a')


In [58]:
#9. rumination01
#NDA: 1 = Almost never; 2 = Sometimes; 3 = Often; 4 = Almost always
questionaire_name = 'rumination01'
results_file = os.path.join(result_dir, f'{questionaire_name}.csv')
template_path = [f for f in glob.glob(os.path.join(approved_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
redcap_file_path = [f for f in glob.glob(os.path.join(redcap_file_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
df_template = pd.read_csv(template_path, header=1)
# Read the first row of template_path with all columns
first_row_template = pd.read_csv(template_path, header=None, nrows=1, names=df_template.columns)
df_redcap = pd.read_csv(redcap_file_path)
df_target_match_names = df_match_names.parse(questionaire_name, usecols=[0, 1])
valid_mappings = df_target_match_names.dropna(subset=['REDCap'])
df = df_template.copy() 
for index, row in valid_mappings.iterrows():
    nda_col = row['NDA_template']  # The destination column name
    redcap_col = row['REDCap']     # The source column name
    if redcap_col in df_redcap.columns:
        df[nda_col] = df_redcap[redcap_col]
# Merge GUID from df_meta to df['subjectkey'] based on matching ID
guid_map = df_meta.set_index('ID')['GUID'].to_dict()
df['src_subject_id'] = df['src_subject_id'].astype(str).str.strip()
df['subjectkey'] = df['src_subject_id'].map(guid_map)
# Merge SEX from df_meta to df['sex'] based on matching ID
sex_map = df_meta.set_index('ID')['SEX'].to_dict()
df['sex'] = df['src_subject_id'].map(sex_map)
# Merge date_of_birth from df_meta into df
dob_map = df_meta.set_index('ID')['date_of_birth'].to_dict()
df['date_of_birth'] = df['src_subject_id'].map(dob_map)
df['interview_age'] = df.apply(
    lambda row: calculate_age_months(row['date_of_birth'], row['interview_date']), 
    axis=1
)
# Convert interview_age to integer
df['interview_age'] = df['interview_age'].astype('Int64')
start_col = 'rumination01'
end_col = 'rumination_total'
cols_to_fix = df.loc[:, start_col:end_col].columns
df[cols_to_fix] = df[cols_to_fix].astype('Int64')

df = df.astype(object)
df = df.where(pd.notnull(df), None)
df = df.dropna(subset=['interview_age', 'interview_date'])
# Convert interview_date to 'MM/DD/YYYY' string format
df['interview_date'] = pd.to_datetime(df['interview_date'])
df['interview_date'] = df['interview_date'].dt.strftime('%m/%d/%Y')
# Trim df to only have columns as in df_template, in the exact same order
df = df[df_template.columns.tolist()]
# Add the first row from template_path to df
df = pd.concat([first_row_template, df], ignore_index=True)
df.iloc[0] = df.iloc[0].fillna('')
df.iloc[:1].to_csv(results_file, index=False, header=False, mode='w')
df.iloc[1:].to_csv(results_file, index=False, header=True, mode='a')

/var/folders/63/3j_hstl96w58qx1sdw9czhxr0000gn/T/ipykernel_54878/4030267616.py:44: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['interview_date'] = pd.to_datetime(df['interview_date'])


In [96]:
#10. image03
questionaire_name = 'image03'
results_file = os.path.join(result_dir, f'{questionaire_name}.csv')
template_path = [f for f in glob.glob(os.path.join(approved_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
defination_path = [f for f in glob.glob(os.path.join(defination_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
df_def = pd.read_csv(defination_path)
dtype_map = dict(zip(df_def['ElementName'], df_def['DataType']))
redcap_file_path = [f for f in glob.glob(os.path.join(redcap_file_dir, '*.csv')) if questionaire_name.lower() in os.path.basename(f).lower()][0]
df_redcap = pd.read_csv(redcap_file_path)
extract_file = os.path.join('/Users/xiaoqianxiao/lab/R01/NDA/sumission_20260115/', 'NDA_image03_extracted.csv')
df_extract = pd.read_csv(extract_file)
df_template = pd.read_csv(template_path, header=1)
# Read the first row of template_path with all columns
first_row_template = pd.read_csv(template_path, header=None, nrows=1, names=df_template.columns)
df = df_template.copy() 
common_cols = df_template.columns.intersection(df_extract.columns)
df = df_extract[common_cols].copy()
# Merge GUID from df_meta to df['subjectkey'] based on matching ID
guid_map = df_meta.set_index('ID')['GUID'].to_dict()
df['src_subject_id'] = df['src_subject_id'].astype(str).str.strip()
df['subjectkey'] = df['src_subject_id'].map(guid_map)
# 1. Ensure IDs are strings for reliable matching
df['src_subject_id'] = df['src_subject_id'].astype(str).str.strip()
df_redcap['record_id'] = df_redcap['record_id'].astype(str).str.strip()

# 2. Optimization: Group REDCap data by ID
redcap_grouped = df_redcap.groupby('record_id')

def get_mri_date(row):
    subject_id = row['src_subject_id']
    
    # Handle NaN visits gracefully and convert to lower case
    visit_val = row['visit']
    if pd.isna(visit_val):
        return row['interview_date']
    
    visit_prefix = str(visit_val).lower()
    
    # Check if this subject exists in REDCap
    if subject_id in redcap_grouped.groups:
        subj_data = redcap_grouped.get_group(subject_id)
        
        # LOGIC: Check startswith ignoring case
        # We convert the entire column to lower(), then check startswith
        match = subj_data[
            subj_data['redcap_event_name']
            .fillna('')
            .str.lower()
            .str.startswith(visit_prefix)
        ]
        
        # If match found, return the mri_exit_date
        if not match.empty:
            # Return the first matching date (or handle duplicates if needed)
            return match.iloc[0]['mri_exit_date']
            
    return row['interview_date']

# 3. Apply the function
df['interview_date'] = df.apply(get_mri_date, axis=1)
# Merge SEX from df_meta to df['sex'] based on matching ID
sex_map = df_meta.set_index('ID')['SEX'].to_dict()
df['sex'] = df['src_subject_id'].map(sex_map)
# Merge date_of_birth from df_meta into df
dob_map = df_meta.set_index('ID')['date_of_birth'].to_dict()
df['date_of_birth'] = df['src_subject_id'].map(dob_map)
df['interview_age'] = df.apply(
    lambda row: calculate_age_months(row['date_of_birth'], row['interview_date']), 
    axis=1
)
# Convert interview_age to integer
df['interview_age'] = df['interview_age'].astype('Int64')
df['experiment_id'] = df['experiment_id'].astype('Int64')
df['image_extent4'] = df['image_extent4'].astype('Int64')


df = df.astype(object)
df = df.where(pd.notnull(df), None)
df = df.dropna(subset=['interview_age', 'interview_date'])
df = df[~df['scan_type'].astype(str).str.contains('localizer scan', case=False, na=False)].copy()
df = df[~df['scan_type'].astype(str).str.contains('MR structural (T2)', case=False, na=False, regex=False)].copy()
# Convert interview_date to 'MM/DD/YYYY' string format
df['interview_date'] = pd.to_datetime(df['interview_date'])
df['interview_date'] = df['interview_date'].dt.strftime('%m/%d/%Y')
# Trim df to only have columns as in df_template, in the exact same order
df = df[df_template.columns.tolist()]
# Add the first row from template_path to df
df = pd.concat([first_row_template, df], ignore_index=True)
df.iloc[0] = df.iloc[0].fillna('')
df.iloc[:1].to_csv(results_file, index=False, header=False, mode='w')
df.iloc[1:].to_csv(results_file, index=False, header=True, mode='a')

/var/folders/63/3j_hstl96w58qx1sdw9czhxr0000gn/T/ipykernel_54878/3464242018.py:83: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['interview_date'] = pd.to_datetime(df['interview_date'])


In [98]:
len(df['src_subject_id'].unique())

50

In [92]:
import os

# Configuration
root_dir = '/Users/xiaoqianxiao/lab/R01/NDA/sumission_20260115/final/image'
target_subfolder = 'anat'
target_tag = 'desc-defaced_T1w'

# SET TO FALSE TO ACTUALLY DELETE
dry_run = True
#dry_run = False

print(f"--- STARTING CLEANUP (Dry Run: {dry_run}) ---")
print(f"Scanning: {root_dir}\n")

cleaned_folders = 0
deleted_files = 0

for dirpath, dirnames, filenames in os.walk(root_dir):
    # Only process directories named 'anat'
    if os.path.basename(dirpath) == target_subfolder:
        cleaned_folders += 1
        print(f"Checking folder: {dirpath}")
        
        # 1. Identify the "Master" files (the defaced NIfTIs)
        # We look for files containing 'desc-defaced_T1w' ending in .nii or .nii.gz
        defaced_niftis = [f for f in filenames if target_tag in f and (f.endswith('.nii') or f.endswith('.nii.gz'))]
        
        # 2. Build the Whitelist (Files we MUST keep)
        whitelist = set()
        
        for nifti in defaced_niftis:
            # Add the image itself to whitelist
            whitelist.add(nifti)
            
            # Derive the corresponding JSON name
            # Logic: Remove 'desc-defaced_' and switch extension to .json
            # e.g., sub-102_..._desc-defaced_T1w.nii.gz -> sub-102_..._T1w.json
            
            # Remove extension first
            if nifti.endswith('.nii.gz'):
                stem = nifti[:-7]
            else:
                stem = nifti[:-4]
            
            # Remove the 'desc-defaced_' tag to match the original JSON
            # We use replace() to handle cases where it might be in the middle
            json_stem = stem.replace('desc-defaced_', '') 
            json_filename = json_stem + '.json'
            
            # If this JSON actually exists, add it to whitelist
            if json_filename in filenames:
                whitelist.add(json_filename)
        
        # 3. Delete everything NOT in the whitelist
        for file in filenames:
            if file not in whitelist:
                full_path = os.path.join(dirpath, file)
                
                if dry_run:
                    print(f"  [WOULD DELETE]: {file}")
                else:
                    os.remove(full_path)
                    print(f"  [DELETED]: {file}")
                deleted_files += 1
            else:
                # Optional: print what we are keeping just to be sure
                # print(f"  [KEEPING]: {file}")
                pass

print(f"\n--- SUMMARY ---")
print(f"Anat folders processed: {cleaned_folders}")
print(f"Files {'marked for deletion' if dry_run else 'deleted'}: {deleted_files}")

if dry_run and deleted_files > 0:
    print(">>> Logic look good? Set 'dry_run = False' and run again.")

--- STARTING CLEANUP (Dry Run: True) ---
Scanning: /Users/xiaoqianxiao/lab/R01/NDA/sumission_20260115/final/image

Checking folder: /Users/xiaoqianxiao/lab/R01/NDA/sumission_20260115/final/image/sub-302/ses-T12/anat
Checking folder: /Users/xiaoqianxiao/lab/R01/NDA/sumission_20260115/final/image/sub-302/ses-repeatbaseline/anat
Checking folder: /Users/xiaoqianxiao/lab/R01/NDA/sumission_20260115/final/image/sub-302/ses-baseline/anat
Checking folder: /Users/xiaoqianxiao/lab/R01/NDA/sumission_20260115/final/image/sub-130/ses-T6/anat
Checking folder: /Users/xiaoqianxiao/lab/R01/NDA/sumission_20260115/final/image/sub-130/ses-repeatbaseline/anat
Checking folder: /Users/xiaoqianxiao/lab/R01/NDA/sumission_20260115/final/image/sub-130/ses-baseline/anat
Checking folder: /Users/xiaoqianxiao/lab/R01/NDA/sumission_20260115/final/image/sub-305/ses-T12/anat
Checking folder: /Users/xiaoqianxiao/lab/R01/NDA/sumission_20260115/final/image/sub-305/ses-repeatbaseline/anat
Checking folder: /Users/xiaoqianxia